# Описание проекта

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В наличии исторические данные: технические характеристики, комплектации и цены автомобилей. Задача - построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

### Инструкция по выполнению проекта

Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: скорость работы, точность результата.

* 		Загрузите и подготовьте данные.
* 		Обучите разные модели. Для каждой попробуйте различные гипепараметеры.
* 		Проанализируйте скорость работы и качество моделей.

#### Примечания:

* 		Для оценки качества моделей применяйте метрику RMSE.
* 		Самостоятельно освойте библиотеку LightGBM и её средствами постройте модели градиентного бустинга.
* 		Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой. Найдите её.
* 		Поскольку модель градиентного бустинга может обучаться долго, измените у неё только два-три параметра.
* 		Если перестанет работать Jupyter Notebook, удалите лишние переменные оператором del:  del features_train 

### Описание данных

Данные находятся в файле /datasets/autos.csv

#### Признаки

* 		DateCrawled — дата скачивания анкеты из базы
* 		VehicleType — тип автомобильного кузова
* 		RegistrationYear — год регистрации автомобиля
* 		Gearbox — тип коробки передач
* 		Power — мощность (л. с.)
* 		Model — модель автомобиля
* 		Kilometer — пробег (км)
* 		RegistrationMonth — месяц регистрации автомобиля
* 		FuelType — тип топлива
* 		Brand — марка автомобиля
* 		NotRepaired — была машина в ремонте или нет
* 		DateCreated — дата создания анкеты
* 		NumberOfPictures — количество фотографий автомобиля
* 		PostalCode — почтовый индекс владельца анкеты (пользователя)
* 		LastSeen — дата последней активности пользователя

#### Целевой признак

Price — цена (евро)


# 1. Подготовка данных

In [1]:
# Импортируем необходимые библитеки и методы
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from scipy import stats as st
from matplotlib.pyplot import show
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

In [2]:
# Читаем исходный файл
data = pd.read_csv('/datasets/autos.csv')

# Смотрим общую информацию
print('INFO')
display(data.info())
print('HEAD 10')
display(data.head(10))

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


None

HEAD 10


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


В исходном датасете присутствуют признаки объективно никак не влияющие на стоимость авто, а именно: `DateCrawled`, `DateCreated`, `NumberOfPictures`, `LastSeen`. Избавимся от них.

In [3]:
data = data.drop(['DateCrawled', 'NumberOfPictures', 'DateCreated', 'LastSeen'], axis=1)
display(data.head(10))

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,70435
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,67112
7,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,19348
8,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,94505
9,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,27472


Изучим исходные данные на предмет пропусков и дубликатов.

In [4]:
def info_func(df):
    print('SPACES')
    display(df.isnull().sum())
    print('DUPLICATES')
    display(df.duplicated().sum())
    print('DIMENSION')
    return df.shape

info_func(data)

SPACES


Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
PostalCode               0
dtype: int64

DUPLICATES


21333

DIMENSION


(354369, 12)

Наблюдается значительное количество пробелов в ряде признаков. Причем все эти признаки - категориальные. Избавимся от пропусков введя новую категорию `unknown`.

In [5]:
col_with_spcs = ['VehicleType','Gearbox', 'Model', 'FuelType', 'NotRepaired']
data[col_with_spcs] = data[col_with_spcs].fillna('unknown')
display(data.head(10))

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,PostalCode
0,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,70435
1,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,67112
7,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,19348
8,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,unknown,94505
9,999,small,1998,manual,101,golf,150000,0,unknown,volkswagen,unknown,27472


In [6]:
info_func(data)

SPACES


Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
PostalCode           0
dtype: int64

DUPLICATES


21333

DIMENSION


(354369, 12)

Избавимся от дубликатов, учитывая их относительно небольшое число по отношению ко всей выборке.

In [7]:
#Удаляем  дубликаты
data = data.drop_duplicates().reset_index(drop = True)

info_func(data)

SPACES


Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
PostalCode           0
dtype: int64

DUPLICATES


0

DIMENSION


(333036, 12)

In [8]:
display(data.describe())

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,PostalCode
count,333036.000000,333036.000000,333036.000000,333036.000000,333036.000000,333036.000000
mean,4378.277586,2004.192268,109.753225,128305.678665,5.694141,50689.740136
std,4502.534823,90.288761,194.288179,37900.007564,3.728757,25804.455970
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,1067.000000
25%,1000.000000,1999.000000,68.000000,125000.000000,3.000000,30179.000000
50%,2699.000000,2003.000000,103.000000,150000.000000,6.000000,49479.000000
75%,6299.250000,2008.000000,140.000000,150000.000000,9.000000,71334.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,99998.000000


Наблюдаются нулевые значения в численных признаках `Price`и `Power`.  Заменить их чем либо не представляется возможным. Удалим записи с нулевыми значениями.

In [9]:
data = data.loc[(data['Power'] > 0)]
data = data.loc[(data['Price'] > 0)]

Также наблюдаются неадекватно большие значения параметра `Power`. Принимая во внимание, что автомобили с мощностью выше чем 1 000 л.с. являются редчайшим явлением (и уж точно продаются не через интернет-площадки) - удалим записи с показателем выше чем 1 000 л.с.

In [10]:
data = data.loc[(data['Power'] <= 1000)]

Представляется разумным также избавиться от "неадекватных" значений признака `RegistrationYear`. Объективно год регистрации автомобиля не может быть ранее 1880 года и позднее 2020.

In [11]:
data = data.loc[(data['RegistrationYear'] < 2021)]
data = data.loc[(data['RegistrationYear'] > 1880)]

Избавимся также от нулевых значений признака `RegistrationMonth`.

In [12]:
data = data.loc[(data['RegistrationMonth'] > 0)]

display(data.describe())

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,PostalCode
count,268347.000000,268347.000000,268347.000000,268347.000000,268347.000000,268347.000000
mean,4942.311831,2003.353796,120.887452,128095.376509,6.372924,51523.623815
std,4624.533768,6.830142,54.754310,36757.153349,3.344437,25743.835277
min,1.000000,1910.000000,1.000000,5000.000000,1.000000,1067.000000
25%,1400.000000,1999.000000,77.000000,125000.000000,4.000000,31157.000000
50%,3300.000000,2003.000000,111.000000,150000.000000,6.000000,50733.000000
75%,7000.000000,2008.000000,150.000000,150000.000000,9.000000,72348.000000
max,20000.000000,2019.000000,1000.000000,150000.000000,12.000000,99998.000000


In [13]:
info_func(data)

SPACES


Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
PostalCode           0
dtype: int64

DUPLICATES


0

DIMENSION


(268347, 12)

Теперь подготовим/закодируем категориальные переменные

In [15]:
# Приеняем технику LabelEncoder
le = preprocessing.LabelEncoder()
le.fit(data.VehicleType)
data['VehicleType_le'] = le.transform(data.VehicleType)
le.fit(data.Gearbox)
data['Gearbox_le'] = le.transform(data.Gearbox)
le.fit(data.Model)
data['Model_le'] = le.transform(data.Model)
le.fit(data.FuelType)
data['FuelType_le'] = le.transform(data.FuelType)
le.fit(data.Brand)
data['Brand_le'] = le.transform(data.Brand)
le.fit(data.NotRepaired)
data['NotRepaired_le'] = le.transform(data.NotRepaired)
le.fit(data.RegistrationMonth)
data['RegistrationMonth_le'] = le.transform(data.RegistrationMonth)
le.fit(data.PostalCode)
data['PostalCode_le'] = le.transform(data.PostalCode)

#Сохраняем все признаки, которые понадобятся для дальнейшего обучения моделей
df = data.drop(['VehicleType', 'Gearbox', 'Model','FuelType', 'Brand', 'NotRepaired',
                'RegistrationMonth','PostalCode'], axis=1)
#Сохраняем список категориальных признаков
cat_features = ['VehicleType_le', 'Gearbox_le', 'Model_le','FuelType_le', 'Brand_le', 
                'NotRepaired_le','RegistrationMonth_le','PostalCode_le']

In [16]:
display(df.head(10))

,Price,RegistrationYear,Power,Kilometer,VehicleType_le,Gearbox_le,Model_le,FuelType_le,Brand_le,NotRepaired_le,RegistrationMonth_le,PostalCode_le
1,18300,2011,190,125000,2,1,227,2,1,2,4,4586
2,9800,2004,163,125000,6,0,117,2,14,1,7,6958
3,1500,2001,75,150000,5,1,116,6,38,0,5,6998
4,3600,2008,69,90000,5,1,101,2,31,0,6,4183
5,650,1995,102,150000,4,1,11,6,2,2,9,2357
6,2200,2004,109,150000,1,1,8,6,25,0,7,4607
8,14500,2014,125,30000,0,1,60,6,10,1,7,7509
10,2000,2004,105,150000,4,1,10,6,19,0,11,7714
11,2799,2005,140,150000,8,1,170,2,38,2,11,4012
12,999,1995,115,150000,8,1,170,6,38,1,10,2663


In [17]:
info_func(df)

SPACES


Price                   0
RegistrationYear        0
Power                   0
Kilometer               0
VehicleType_le          0
Gearbox_le              0
Model_le                0
FuelType_le             0
Brand_le                0
NotRepaired_le          0
RegistrationMonth_le    0
PostalCode_le           0
dtype: int64

DUPLICATES


0

DIMENSION


(268347, 12)

# 1. Подготовка данных. Выводы.

- исходный датасет представлен 354 369 записями и 16 признаками
- названия столбцов информативны и удобочитаемы
- были удалены, как не влияющие на стоимость авто, признаки: `DateCrawled`, `DateCreated`, `NumberOfPictures`, `LastSeen`
- пропущенные значения присутствовали только в категориальных признаках и были заменены на новую категорию `unknown`
- дубликаты были удалены, в связи с их небольшим количеством, относительно общего объема выборки
- удалены явные аномалии и ошибки: нулевые значения(цена, мощность, месяц регистрации), аномальные значения года регистрации и мощности
- произведено масштабирование количественных признаков `Power` и `Kilometer`
- осуществлено кодирование категориальных признаков техникой LabelEncoder
- размерность конечного датасета составляет 268 347 записей и 12 признаков

# 2. Обучение моделей

### 2.1 Создание обучающей, валидационной и  тестовой выборок

In [18]:
# Проведем последовательное разбиение. Сначала разделим датасет на обучающую и валидационную выборки (70% отдаем обучающей)
df_train, df_valid = train_test_split(df, test_size = 0.3, random_state = 12345)

# Теперь поделим пополам валидационную выборку, тем самым создав тестовую
df_valid, df_test = train_test_split(df_valid, test_size = 0.5, random_state=12345)

# Контрольная проверка размерности выборок
print('Размерность обучащей выборки:', df_train.shape)
print('Размерность валидационной выборки:', df_valid.shape)
print('Размерность тестовой выборки:', df_test.shape)

Размерность обучащей выборки: (187842, 12)
Размерность валидационной выборки: (40252, 12)
Размерность тестовой выборки: (40253, 12)


В связи с различными масштабами значений, проведем нормализацию численных признаков `Power` и `Kilometer`

In [19]:
def normalization(frame, columns):
    numeric_cols = columns
    features = frame[numeric_cols]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    frame[numeric_cols] = features

normalization(df_train,['Power', 'Kilometer'])
normalization(df_valid,['Power', 'Kilometer'])
normalization(df_test,['Power', 'Kilometer'])

In [23]:
# Подготовим переменные для признаков и целевого признака различных выборок
features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']

### 2.2 Обучение модели алгоритмом CatBoost (перебор в цикле 2 гиперпараметров)

#### Обучение

In [24]:
%%time

result = pd.DataFrame(columns=['max_depth','max_estimators', 'RMSE_CB'])
print("Идет обучение модели на базе алгоритма CatBoost ...")

i=0
for max_depth in range(1,17,5):
    for max_estimators in range(1,102,25):
        model_CB = CatBoostRegressor(loss_function="RMSE", max_depth = max_depth, n_estimators= max_estimators)
        model_CB.fit(features_train, target_train, cat_features=cat_features, verbose= 20)
        predicted_CB_valid = model_CB.predict(features_valid)
        rmse_CB = sqrt(mean_squared_error(target_valid, predicted_CB_valid))
        result.loc[i] = [max_depth, int(max_estimators), int(rmse_CB)]
        print(result.loc[i])
        i+=1

Идет обучение модели на базе алгоритма CatBoost ...
0:	learn: 4576.9873858	total: 226ms	remaining: 0us
max_depth            1
max_estimators       1
RMSE_CB           4561
Name: 0, dtype: object
0:	learn: 4576.9873858	total: 78.4ms	remaining: 1.96s
20:	learn: 3982.8035877	total: 2.27s	remaining: 540ms
25:	learn: 3882.4010270	total: 2.76s	remaining: 0us
max_depth            1
max_estimators      26
RMSE_CB           3862
Name: 1, dtype: object
0:	learn: 4576.9873858	total: 74.7ms	remaining: 3.73s
20:	learn: 3982.8035877	total: 2.46s	remaining: 3.51s
40:	learn: 3643.5957989	total: 4.45s	remaining: 1.09s
50:	learn: 3517.0073058	total: 5.54s	remaining: 0us
max_depth            1
max_estimators      51
RMSE_CB           3492
Name: 2, dtype: object
0:	learn: 4576.9873858	total: 46.3ms	remaining: 3.47s
20:	learn: 3982.8035877	total: 2.14s	remaining: 5.59s
40:	learn: 3643.5957989	total: 4.24s	remaining: 3.62s
60:	learn: 3409.0244322	total: 6.31s	remaining: 1.55s
75:	learn: 3274.4823474	total: 

In [25]:
print('Результаты обучения алгоритмом CatBoost', end = '\n')
result.sort_values('RMSE_CB')

Результаты обучения алгоритмом CatBoost


,max_depth,max_estimators,RMSE_CB
19,16,101,1718
14,11,101,1797
18,16,76,1817
13,11,76,1899
9,6,101,1992
17,16,51,2076
8,6,76,2128
12,11,51,2152
7,6,51,2402
16,16,26,2771


С точки зрения оптимальности (скорость и значение RMSE) наилучшей комбинацией гиперпараметров является `depth` = 11 и `n_estimators`= 101 (время обучения - 1 мин., значение RMSE = 1 797)

#### Проверка на валидационной выборке оптимальной комбинации гиперпараметров

In [26]:
%%time

print("Идет обучение модели на базе алгоритма CatBoost...")
model_CB = CatBoostRegressor(loss_function="RMSE", max_depth =11, n_estimators=101)
model_CB.fit(features_train, target_train, cat_features=cat_features, verbose= 20)
predicted_CB_valid = model_CB.predict(features_valid)
rmse_CB = sqrt(mean_squared_error(target_valid, predicted_CB_valid))

Идет обучение модели на базе алгоритма CatBoost...
0:	learn: 4521.9923069	total: 650ms	remaining: 1m 5s
20:	learn: 3073.7311636	total: 12.8s	remaining: 48.9s
40:	learn: 2361.6223842	total: 24.8s	remaining: 36.3s
60:	learn: 2034.7697043	total: 36.7s	remaining: 24.1s
80:	learn: 1881.4341872	total: 48.6s	remaining: 12s
100:	learn: 1801.7156397	total: 1m	remaining: 0us
CPU times: user 57.3 s, sys: 4.83 s, total: 1min 2s
Wall time: 1min 3s


In [27]:
print('\n')
print('Значение RMSE(CatBoost) на валидационной выборке:', int(rmse_CB))



Значение RMSE(CatBoost) на валидационной выборке: 1797


### 2.2 Обучение модели алгоритмом LightGBM (перебор в цикле 2 гиперпараметров)

#### Обучение

In [28]:
%%time

result = pd.DataFrame(columns=['max_depth','max_estimators', 'RMSE_LGBM'])
print("Идет обучение модели на базе алгоритма LightGBM ...")

i=0
for max_depth in range(1,17,5):
    for max_estimators in range(1,102,25):
        model_LGBM = LGBMRegressor(loss_function="RMSE", max_depth = max_depth, n_estimators= max_estimators)
        model_LGBM.fit(features_train, target_train)
        predicted_LGBM_valid = model_LGBM.predict(features_valid)
        rmse_LGBM = sqrt(mean_squared_error(target_valid, predicted_LGBM_valid))
        result.loc[i] = [max_depth, int(max_estimators), int(rmse_LGBM)]
        print(result.loc[i])
        i+=1

Идет обучение модели на базе алгоритма LightGBM ...
max_depth            1
max_estimators       1
RMSE_LGBM         4457
Name: 0, dtype: object
max_depth            1
max_estimators      26
RMSE_LGBM         3168
Name: 1, dtype: object
max_depth            1
max_estimators      51
RMSE_LGBM         2803
Name: 2, dtype: object
max_depth            1
max_estimators      76
RMSE_LGBM         2613
Name: 3, dtype: object
max_depth            1
max_estimators     101
RMSE_LGBM         2504
Name: 4, dtype: object
max_depth            6
max_estimators       1
RMSE_LGBM         4279
Name: 5, dtype: object
max_depth            6
max_estimators      26
RMSE_LGBM         1982
Name: 6, dtype: object
max_depth            6
max_estimators      51
RMSE_LGBM         1804
Name: 7, dtype: object
max_depth            6
max_estimators      76
RMSE_LGBM         1752
Name: 8, dtype: object
max_depth            6
max_estimators     101
RMSE_LGBM         1725
Name: 9, dtype: object
max_depth           11
max_e

In [29]:
print('Результаты обучения алгоритмом LightGBM', end = '\n')
result.sort_values('RMSE_LGBM')

Результаты обучения алгоритмом LightGBM


,max_depth,max_estimators,RMSE_LGBM
14,11,101,1700
19,16,101,1704
13,11,76,1725
9,6,101,1725
18,16,76,1729
8,6,76,1752
12,11,51,1778
17,16,51,1782
7,6,51,1804
11,11,26,1965


Наилучшей комбинацией гиперпараметров является `depth` = 11 и `n_estimators`= 101 (значение RMSE = 1 700)

#### Проверка на валидационной выборке оптимальной комбинации гиперпараметров

In [30]:
%%time

print("Идет обучение модели на базе алгоритма LightGBM...")
model_LGBM = LGBMRegressor(loss_function="RMSE", max_depth =11, n_estimators=101)
model_LGBM.fit(features_train, target_train)
predicted_LGBM_valid = model_LGBM.predict(features_valid)
rmse_LGBM = sqrt(mean_squared_error(target_valid, predicted_LGBM_valid))

Идет обучение модели на базе алгоритма LightGBM...
CPU times: user 7.01 s, sys: 46.8 ms, total: 7.06 s
Wall time: 7.11 s


In [31]:
print('\n')
print('Значение RMSE(LightGBM) на валидационной выборке:', int(rmse_LGBM))



Значение RMSE(LightGBM) на валидационной выборке: 1700


### 2.3 Обучение модели алгоритмом LinearRegression

Для сравнения проанализируем результаты применения одного из простейших алгоритмов - LinearRegression

#### Обучение

In [32]:
%%time

model_LR = LinearRegression()
model_LR.fit(features_train, target_train)
predicted_LR_train = model_LR.predict(features_train)
rmse_LR = sqrt(mean_squared_error(target_train, predicted_LR_train))

CPU times: user 94.6 ms, sys: 50 ms, total: 145 ms
Wall time: 105 ms


In [33]:
print('RMSE (LinearRegression):',int(rmse_LR))

RMSE (LinearRegression): 2977


#### Проверка на валидационной выборке

In [34]:
%%time

model_LR.fit(features_train, target_train)
predicted_LR_valid = model_LR.predict(features_valid)
rmse_LR = sqrt(mean_squared_error(target_valid, predicted_LR_valid))

CPU times: user 73.3 ms, sys: 28 ms, total: 101 ms
Wall time: 99.2 ms


In [35]:
print('\n')
print('Значение RMSE(LinearRegression) на валидационной выборке:', int(rmse_LR))



Значение RMSE(LinearRegression) на валидационной выборке: 2964


RMSE значительно хуже чем у алгоритмов градиентного бустинга

# 2. Обучение моделей. Выводы.

- для обучения были применены алгоритмы: `CatBoost`, `LightGBM`
- выбраны оптимальные, с точки зрения скорости обучения и точности, гиперпараметры. А именно:
    - для `CatBoost`: `depth` = 11, `n_estimators` = 101
    - для `LightGBM`: `depth` = 11, `n_estimators` = 101

# 3. Анализ моделей

#### Применяем обученную на оптимальных гиперпараметрах, алгоритмом CatBoost, модель к тестовой выборке

In [36]:
%%time

predict_CB_test = model_CB.predict(features_test)
rmse_CB = sqrt(mean_squared_error(target_test, predict_CB_test))
print('RMSE(CatBoost) на тестовой выборке:',int(rmse_CB), '\n')

RMSE(CatBoost) на тестовой выборке: 1842 

CPU times: user 210 ms, sys: 13.4 ms, total: 223 ms
Wall time: 208 ms


#### Применяем обученную на оптимальных гиперпараметрах, алгоритмом LightGBM, модель к тестовой выборке

In [37]:
%%time

pred_LGBM_test = model_LGBM.predict(features_test)
rmse_LGBM = sqrt(mean_squared_error(target_test, pred_LGBM_test))
print('RMSE(LightGBM) на тестовой выборке:',int(rmse_LGBM), '\n')

RMSE(LightGBM) на тестовой выборке: 1735 

CPU times: user 471 ms, sys: 0 ns, total: 471 ms
Wall time: 490 ms


#### Применяем обученную, алгоритмом LinearRegression, модель к тестовой выборке

In [38]:
%%time

pred_LR_test = model_LR.predict(features_test)
rmse_LR = sqrt(mean_squared_error(target_test, pred_LR_test))
print('RMSE(LinearRegression) на тестовой выборке:',int(rmse_LR), '\n')

RMSE(LinearRegression) на тестовой выборке: 3006 

CPU times: user 7.25 ms, sys: 9.84 ms, total: 17.1 ms
Wall time: 4.81 ms


#### Сравниваем поученные метрики RMSE с метрикой модели, где все значения средние по целевому признаку

In [39]:
%%time

mean = target_train.mean()
lenght = features_test.shape[0]
const_predictions = np.full((lenght,1), mean)
const_rmse = sqrt(mean_squared_error(const_predictions, target_test))
print('RMSE(константная модель) на тестовой выборке', int(const_rmse), '\n')

RMSE(константная модель) на тестовой выборке 4649 

CPU times: user 15.3 ms, sys: 1.48 ms, total: 16.8 ms
Wall time: 2.5 ms


# 3. Анализ моделей. Выводы.

- выбраны оптимальные, с точки зрения скорости обучения и точности, гиперпараметры. А именно:
    - для `CatBoost`: `depth` = 11, `n_estimators` = 101 (скорость обучения - 1 мин 02 сек., RMSE = 1 797)
    - для `LightGBM`: `depth` = 11, `n_estimators` = 101 (скорость обучения - 7,06 сек., RMSE = 1 700)
- алгоритм `CatBoost` способен на большую точность, но скорость обучения при этом значительно снижается
- для сравнения были проанализированы результаты работы одного из простейших алгоритмов - `LinearRegression` Алгоритм показал значительно худший показатель RMSE = 2 964
- на тестовой выборке лучший результат показал алгоритм `LightGBM` (RMSE = 1 735)
- результат алгоритма `CatBoost` чуть хуже (RMSE = 1 842)
- результат алгоритма `LinearRegression` самый низкий из тестируемых (RMSE = 3 006)
- обе модели градиентного бустинга работают значительно лучше чем константная модель на основе средних значений (RMSE = 4 649)

# Общий вывод.


### Цель исследования

На основании имеющегося датасета с техническими характеристиками автомобилей построить модель, предсказывающую стоимость автомобиля по его тех.характеристикам с оптимальной скоростью и качеством предсказания.

### Описание исследования

- для исследования был представлен датасет с 16 признаками (среди них "Price" - целевой) и 354 369 атрибутами
- при подготовке исходных данных были удалены пропуски, дубликаты, аномальные выбросы по числовым признакам
- при подготовке датасета категориальные признаки были переведены в формат `int`, с помощью метода preprocessing.LabelEncoder() библиотеки sclearn
- численные признаки были масштабированы
- для обучения модели были выбраны алгоритмы градиентного бустинга `CatBoost`, `LightGBM` и `LinearRegression`
- для выбора оптимальных значений гиперпарамтров, для обоих алгоритмов градиентного бустинга, было организовано обучение модели посредством циклов
- реальное время работы алгоритмов, в заданных циклах для гиперпараметров, на тренинговой выборке составило:
    - CatBoost = 18 мин 11 сек
    - LightGBM = 1 мин 08 сек
    - LinearRegression = 0,145 сек
- проведено сравнение эффективности работы моделей с константной моделью на основе среднего значения целевого признака
    
### Результаты

В результате применения оптимальных значений для гиперпараметров были получены следующие результаты на тестовой выборке:

- LightGBM: depth = 11, n_estimators = 101, RMSE LightGBM = 1 735, real running time= 0,471 сек
- CatBoost: depth = 11, n_estimators = 101, RMSE CatBoost = 1 842, real running time= 0,223 сек
- LinearRegression: RMSE LinearRegression = 3 006, real running time= 0,017 сек

Таким образом, наиболее подходящим, с учетом требований Заказчика относительно оптимальной скорости и точности, является алгоритм `LightGBM`, как более точный и значительно более быстрый в плане времени обучения. Алгоритм `CatBoost` сопоставим по точности, но объективно проигрывает в плане времени обучения.


===================================================================================================================

#### Благодарю за внимание.
